# 06_FEP_Subtraction_SSA

### Developed by SSA
for the subtraction of a known contaminant data set from experimental PXRD. in this case it is removal of FEP peaks from a crystal powder pattern where collection was complete using FEP windows 

## Imports 

In [ ]:
#imports & helper functions
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
# Try importing scipy.signal utilities; fall back to numpy versions if not installed
try:
    from scipy.signal import correlate, find_peaks
except Exception:
    correlate = None
    find_peaks = None
    print("Note: scipy.signal not available — using numpy fallback for cross-correlation. "
          "If you want better peak finding/xcorr, install scipy with `conda install -c conda-forge scipy`.")

# small helper: robust file loader that skips text headers
def robust_loadtxt(path, expected_cols=2, max_header_lines=10):
    """
    Try to load numeric data. If first attempt fails because of headers, scan first
    max_header_lines lines to find first numeric line and use skiprows accordingly.
    Returns an (N, M) numpy array.
    """
    # try direct
    try:
        data = np.loadtxt(path)
        return data
    except Exception:
        # inspect file to find first numeric line
        with open(path, 'r', errors='ignore') as f:
            lines = [next(f) for _ in range(max_header_lines)]
        skip = 0
        for i, ln in enumerate(lines):
            parts = ln.strip().split()
            # consider numeric if we can convert required number of columns to float
            if len(parts) >= expected_cols:
                try:
                    [float(x) for x in parts[:expected_cols]]
                    skip = i
                    break
                except Exception:
                    continue
        # if skip == 0 it may still have header starting at line 0; attempt skiprows=1..max_header_lines
        for try_skip in range(skip, max_header_lines+1):
            try:
                data = np.loadtxt(path, skiprows=try_skip)
                return data
            except Exception:
                continue
    raise ValueError(f"Could not parse numeric data from {path} with up to {max_header_lines} header lines.")


## File Location and Output

In [ ]:
#make output directory 
processing_folder_dir = "D:/I11 Beamtime July/RAW_2D/Run_5_X3_0.5VF/"  

processing_folder = "06_FEP_Subtracted/"
processing_path = os.path.join(processing_folder_dir ,processing_folder)


if not os.path.exists(processing_path):
    os.makedirs(processing_path)


print("Processed pattern directory:", processing_path)

In [ ]:
#load files (edit paths if needed)
processed_path = r"D:/I11 Beamtime July/RAW_2D/Run_5_X3_0.5VF/04B_PM_Baselined_Merged/Run_5_X3_0.5VF_final_merged_pattern.xy" #('C:/fep/ conatminated/ file/.xy')
fep_path       = r"D:/I11 Beamtime July/RAW_2D/FEP_background_subtraction.xye" #('C:/fep/background/scan/.xy')

# load with robust loader
processed_data = robust_loadtxt(processed_path)
fep_data = robust_loadtxt(fep_path)

# Accept either 2 or 3 columns in fep_data; if 3, ignore errors column
if processed_data.shape[1] < 2:
    raise ValueError("Processed file did not contain at least 2 columns.")
if fep_data.shape[1] < 2:
    raise ValueError("FEP file did not contain at least 2 columns.")

two_theta_data = processed_data[:,0]
intensity_data = processed_data[:,1]

two_theta_FEP = fep_data[:,0]
intensity_FEP = fep_data[:,1]

print("Loaded data shapes:")
print(f" - processed: {processed_data.shape}")
print(f" - FEP:       {fep_data.shape}")
print(f"two_theta range (processed): {two_theta_data.min():.3f} — {two_theta_data.max():.3f}")
print(f"two_theta range (FEP):       {two_theta_FEP.min():.3f} — {two_theta_FEP.max():.3f}")


## Initial Combined Plot 

In [ ]:
# Initial Combined Plot (processed, interpolated FEP, raw subtraction)
# interpolate FEP to data axis
fep_interp_full = np.interp(two_theta_data, two_theta_FEP, intensity_FEP)

# raw subtraction (just to visualise)
raw_sub = intensity_data - fep_interp_full

plt.figure(figsize=(10,6))
plt.plot(two_theta_data, intensity_data, label='Processed data', lw=1)
plt.plot(two_theta_data, fep_interp_full, label='FEP (interp)', lw=1)
plt.plot(two_theta_data, raw_sub, label='Raw subtraction', lw=1.2)
plt.xlabel("2θ (°)")
plt.ylabel("Intensity (a.u.)")
plt.title("Combined: processed, FEP (interp), raw subtraction")
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print("Raw subtraction: min = {:.3g}, max = {:.3g}".format(raw_sub.min(), raw_sub.max()))


## Gaussian Fit and FEP Removal 

In [ ]:
#FEP removal via Gaussian fitting 
from scipy.optimize import curve_fit

#Parameters
#this is incase the FEP peak is so intense that still causes issues even after automatoc subtraction - generally needed to get rid of one strong peak 
fep_peak_min = 5.6  # lower 2θ bound of FEP peak (adjust as needed)
fep_peak_max = 5.8  # upper 2θ bound of FEP peak (adjust as needed)



#1. Mask FEP region for fitting
window_mask = (two_theta_data >= fep_peak_min) & (two_theta_data <= fep_peak_max)
x_window = two_theta_data[window_mask]
y_window = intensity_data[window_mask]

#2. Define Gaussian function
def gauss(x, A, x0, sigma):
    return A * np.exp(-(x - x0)**2 / (2 * sigma**2))

#3. Initial guess for fitting parameters
A_guess = np.max(y_window)
x0_guess = x_window[np.argmax(y_window)]
sigma_guess = 0.05  # adjust if FEP peak is broader/narrower
p0 = [A_guess, x0_guess, sigma_guess]

#4. Fit Gaussian to FEP peak
try:
    popt, _ = curve_fit(gauss, x_window, y_window, p0=p0)
    print(f"Fitted FEP parameters: Amplitude={popt[0]:.2f}, Center={popt[1]:.4f}°, Sigma={popt[2]:.4f}°")
except RuntimeError:
    print("Gaussian fit failed; using raw FEP subtraction instead.")
    popt = None

#5. Construct fitted FEP over full 2θ range
if popt is not None:
    fep_fit_full = gauss(two_theta_data, *popt)
else:
    # fallback: use original intensity in window, zero elsewhere
    fep_fit_full = np.zeros_like(intensity_data)
    fep_fit_full[window_mask] = intensity_data[window_mask]

# 6. Subtract FEP from experimental data
data_minus_FEP = intensity_data - fep_fit_full
data_minus_FEP[data_minus_FEP < 0] = 0  # optional: clip negative intensities

# 7. Plot results
plt.figure(figsize=(10,5))
plt.plot(two_theta_data, intensity_data, label='Original data', alpha=0.5)
plt.plot(two_theta_data, fep_fit_full, label='Fitted FEP peak', lw=1)
plt.plot(two_theta_data, data_minus_FEP, label='Data - FEP', lw=1)
plt.legend()
plt.title("FEP removal via Gaussian fitting (weak peaks preserved)")
plt.xlabel("2θ (°)")
plt.ylabel("Intensity (a.u.)")
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()



## Scaling and Single Graph Production 

In [ ]:
#detect main FEP peak region and safely compute local scaling

# 1. Use the Gaussian-fitted FEP from previous cell
FEP_for_scaling = fep_fit_full  # the fitted FEP

# Peak detection: fallback to simple argmax if scipy.find_peaks not available
if find_peaks is not None:
    threshold = np.max(FEP_for_scaling) * 0.5
    peaks, props = find_peaks(FEP_for_scaling, height=threshold)
    if len(peaks) == 0:
        peaks, props = find_peaks(FEP_for_scaling, height=np.max(FEP_for_scaling)*0.3)
else:
    peaks = np.array([np.argmax(FEP_for_scaling)])



# 2. Determine main peak index
if len(peaks) == 0:
    main_idx = int(np.argmax(FEP_for_scaling))
else:
    main_idx = peaks[np.argmax(FEP_for_scaling[peaks])]

peak_pos = two_theta_data[main_idx]
print(f"Detected main FEP peak at {peak_pos:.3f}° (index {main_idx})")



# 3. Define region width around main peak for scaling
region_width = 0.5  # degrees
mask_region = (two_theta_data >= (peak_pos - region_width)) & (two_theta_data <= (peak_pos + region_width))



# 4. Compute least-squares scale in the local region
num = np.sum(intensity_data[mask_region] * FEP_for_scaling[mask_region])
den = np.sum(FEP_for_scaling[mask_region] ** 2)
scale_local = (num / den) if den != 0 else 0.0
print(f"Local scale factor (region ±{region_width}°) = {scale_local:.6g}")



# 5. Subtract FEP only in the peak region and clip negatives
sub_local = intensity_data.copy()
sub_local[mask_region] -= FEP_for_scaling[mask_region] * scale_local
sub_local[sub_local < 0] = 0  # avoid negative intensities <<<--- this prevents the baseline from being negative 

# 6. Plot results
plt.figure(figsize=(10,6))
plt.plot(two_theta_data, intensity_data, label='Processed data', lw=1)
plt.plot(two_theta_data, FEP_for_scaling * scale_local, label=f'FEP scaled (s={scale_local:.4g})', lw=1)
plt.plot(two_theta_data, sub_local, label='Subtraction (clipped)', lw=1.2)
plt.xlim(peak_pos - 2.0, peak_pos + 2.0)
plt.legend()
plt.title("Local FEP subtraction (safe & clipped)")
plt.xlabel("2θ (°)")
plt.ylabel("Intensity (a.u.)")
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()


In [ ]:
#Apply global scale, subtract FEP, and safe baseline correction

# 1. manual scale is needed 
manual_scale = None   # <-- set a manual factor if needed, e.g., 0.95
scale_to_use = float(manual_scale) if manual_scale is not None else float(scale_local)
print(f"Using scale factor = {scale_to_use:.6g}")


# 2. Use Gaussian-fitted FEP instead of old shifted_FEP
scaled_fep_global = fep_fit_full * scale_to_use


# 3.  Subtract globally
subtracted_global = intensity_data - scaled_fep_global


# 4. baseline correction 
# Estimate baseline as the 5th percentile
baseline_shift = np.percentile(subtracted_global, 5)

# Shift pattern so baseline sits at ~0
subtracted_corrected = subtracted_global - baseline_shift

# Clip any negative values to 0 to avoid baseline going below zero
subtracted_corrected[subtracted_corrected < 0] = 0

print(f"Applied baseline shift = {baseline_shift:.4g} (clipped negatives to 0)")



# 5. Plot results 
plt.figure(figsize=(10,6))
plt.plot(two_theta_data, intensity_data, label='Processed data', alpha=0.7)
plt.plot(two_theta_data, scaled_fep_global, label='Scaled FEP (Gaussian fit)', alpha=0.7)
plt.plot(two_theta_data, subtracted_corrected, label='Final subtracted (baseline ~0)', lw=1.3, color='black')
plt.xlabel("2θ (°)")
plt.ylabel("Intensity (a.u.)")
plt.title("Final FEP subtraction (global application of local scale)")
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()



# 6. Save figure BEFORE show
out_png = os.path.join(processing_path, "FEP_subtracted_final.png")
plt.savefig(out_png, dpi=300)
plt.show()

print("Final: raw-sub min/max = {:.3g}/{:.3g}".format(subtracted_global.min(), subtracted_global.max()))
print("After baseline correction & clipping min/max = {:.3g}/{:.3g}".format(subtracted_corrected.min(), subtracted_corrected.max()))
print(f"Saved diagnostic figure to: {out_png}")



In [ ]:
#Save outputs (updated for Gaussian-fitted FEP workflow)

save_dir = os.path.join(processing_path, "06_fep_subtraction_results")
os.makedirs(save_dir, exist_ok=True)

xy_final = os.path.join(save_dir, "FEP_subtracted_final_baseline0.xy")
xy_raw   = os.path.join(save_dir, "FEP_subtracted_final_raw.xy")
xy_cont  = os.path.join(save_dir, "FEP_contaminant_fitted_scaled.xy")
png_out  = os.path.join(save_dir, "FEP_subtracted_final_baseline0.png")  # consistent name

# Save baseline-corrected final (clipped & baseline=0)
np.savetxt(xy_final, np.c_[two_theta_data, subtracted_corrected],
           header=f"2Theta Intensity (FEP-subtracted, baseline=0, scale={scale_to_use:.6g})", comments='')

# Save raw subtracted (before baseline correction/clipping) for record
np.savetxt(xy_raw, np.c_[two_theta_data, subtracted_global],
           header=f"2Theta Intensity (FEP-subtracted, raw, scale={scale_to_use:.6g})", comments='')

# Save the fitted and globally scaled FEP (used for subtraction)
np.savetxt(xy_cont, np.c_[two_theta_data, scaled_fep_global],
           header=f"2Theta Intensity (Gaussian-fitted FEP, scaled by {scale_to_use:.6g})", comments='')

# Re-save PNG of final trace
plt.figure(figsize=(9,5))
plt.plot(two_theta_data, subtracted_corrected, color='black', lw=1.2)
plt.xlabel("2θ (°)")
plt.ylabel("Intensity (a.u.)")
plt.xlim(1,30)
plt.ylim(-10,800)
plt.title("FEP-subtracted final (baseline = 0)")
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig(png_out, dpi=300)
plt.show()

print("Saved files:")
print(" - final baseline-corrected .xy:", xy_final)
print(" - raw subtracted (unclipped) .xy:", xy_raw)
print(" - fitted & scaled FEP .xy:", xy_cont)
print(" - final PNG:", png_out)
